### the features for mixed voices are extracted.

In [1]:
import numpy as np
import os
from scipy.io import wavfile
import librosa
import csv
import math as m
from natsort import natsorted
from sklearn import preprocessing
import h5py
import soundfile as sf

#### paths are defined.

In [2]:
orig_path=os.getcwd()
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/30hdata'
mixed_folder = os.path.normpath(os.path.join(write_path,'mixed_30h'))
clean_raw = 'clean_30h'
w=7

mixed_log_30h and mixed_phase_30h are the log and phase of mixed files with some zero elements in between to make the ftr_refrmd file which is the same thing but with context.

they're both stored as h5py file.

In [3]:
a = os.listdir(os.path.join(mixed_folder))
a = natsorted(a)

mixed_logname = 'mixed_log_30h'
mixed_phasename = 'mixed_phase_30h'
mixed_logpath = os.path.normpath(os.path.join(write_path,mixed_logname))
mixed_phasepath = os.path.normpath(os.path.join(write_path,mixed_phasename))

f = h5py.File(mixed_logpath+'.hdf5', 'w')
d = f.create_dataset(mixed_logname, (0,257),maxshape=(None,257), dtype='f', chunks=True)
ff = h5py.File(mixed_phasepath+'.hdf5', 'w')
dd = ff.create_dataset(mixed_phasename, (0,257),maxshape=(None,257), dtype='f', chunks=True)
for filename in a:
    wav_data, sr = librosa.load(os.path.join(mixed_folder,filename), sr=16000) 
    framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
    abslt=np.absolute(framed_data)**2
    dft_signal=np.log10(abslt+(1e-7*np.ones(np.shape(abslt))))
    for i in range(m.floor(w/2)):
        dft_signal=np.insert(dft_signal,0,0,axis = 1)
    data_phase=np.angle(framed_data)
    for i in range(m.floor(w/2)):
        data_phase=np.insert(data_phase,0,0,axis = 1)
    d.resize(d.shape[0]+dft_signal.shape[1], axis=0)   
    d[-1*dft_signal.shape[1]:] = dft_signal.T
    dd.resize(dd.shape[0]+data_phase.shape[1], axis=0)   
    dd[-1*data_phase.shape[1]:] = data_phase.T

f.close()
ff.close()

In [11]:
f.close()
ff.close()

mixed_log_30h_nozeroinsert and mixed_phase_30h_nozeroinsert are the log and phase of mixed files without zero elements in between.

they're both stored as h5py file.

In [4]:
a = os.listdir(os.path.join(mixed_folder))
a = natsorted(a)

mixed_logname = 'mixed_log_30h_nozeroinsert'
mixed_phasename = 'mixed_phase_30h_nozeroinsert'
mixed_logpath = os.path.normpath(os.path.join(write_path,mixed_logname))
mixed_phasepath = os.path.normpath(os.path.join(write_path,mixed_phasename))

f = h5py.File(mixed_logpath+'.hdf5', 'w')
d = f.create_dataset(mixed_logname, (0,257),maxshape=(None,257), dtype='f', chunks=True)
ff = h5py.File(mixed_phasepath+'.hdf5', 'w')
dd = ff.create_dataset(mixed_phasename, (0,257),maxshape=(None,257), dtype='f', chunks=True)

for filename in a:
    wav_data, sr = librosa.load(os.path.join(mixed_folder,filename), sr=16000) 
    framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
    abslt=np.absolute(framed_data)**2
    dft_signal=np.log10(abslt+(1e-7*np.ones(np.shape(abslt))))
    data_phase=np.angle(framed_data)
    d.resize(d.shape[0]+dft_signal.shape[1], axis=0)   
    d[-1*dft_signal.shape[1]:] = dft_signal.T
    dd.resize(dd.shape[0]+data_phase.shape[1], axis=0)   
    dd[-1*data_phase.shape[1]:] = data_phase.T
    i=i+1
f.close()
ff.close()

'single_dataset_log_30h' and 'single_dataset_phase_30h'are log and phase of clean data.
they are both without zero elements.

In [5]:
num_of_snrs=21
num_of_speakers=6

files = os.listdir(os.path.normpath(os.path.join(write_path,clean_raw)))
files=natsorted(files)

clean_log = 'single_dataset_log_30h'
clean_phase = 'single_dataset_phase_30h'
clean_log_path = os.path.normpath(os.path.join(write_path,clean_log))
clean_phase_path = os.path.normpath(os.path.join(write_path,clean_phase))

f = h5py.File(clean_log_path+'.hdf5', 'w')
d = f.create_dataset(clean_log, (0,257),maxshape=(None,257), dtype='f', chunks=True)
ff = h5py.File(clean_phase_path+'.hdf5', 'w')
dd = ff.create_dataset(clean_phase, (0,257),maxshape=(None,257), dtype='f', chunks=True)

for voice in files:
    wav_data, sr = librosa.load(os.path.join(write_path,clean_raw,voice), sr=16000) 
    framed_data=librosa.core.stft(wav_data, n_fft=512, hop_length=256, win_length=512, window='hann')
    abslt=np.absolute(framed_data)**2
    dft_signal=np.log10(abslt+(1e-7*np.ones(np.shape(abslt))))
    data_phase=np.angle(framed_data)
#     for i in range(num_of_snrs*num_of_speakers):
    d.resize(d.shape[0]+dft_signal.shape[1], axis=0)   
    d[-1*dft_signal.shape[1]:] = dft_signal.T
    dd.resize(dd.shape[0]+data_phase.shape[1], axis=0)   
    dd[-1*data_phase.shape[1]:] = data_phase.T
f.close()
ff.close()


'ftr_refrmd_30h' is the refrmd(contexted) version of mixed_log_30h.

In [ ]:
mixed_logname='mixed_log_30h'
mixed_phasename='mixed_phase_30h'
mixed_log_path = os.path.normpath(os.path.join(write_path,mixed_logname))
h5f = h5py.File(mixed_log_path+'.hdf5','r')
ftr = h5f[mixed_logname][0:]

ftr_refrmd=[]
for i in range(m.floor(w/2)):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        temp.extend(ftr[i+p+j])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
        
refrmd_file='ftr_refrmd_30h'
refrmd_file_path = os.path.normpath(os.path.join(write_path,refrmd_file))
f = h5py.File(refrmd_file_path+'.hdf5', 'w')
d = f.create_dataset(refrmd_file, (0,257*w),maxshape=(None,257*w), dtype='f', chunks=True)

for i in range(m.floor(w/2),ftr.shape[0]-m.floor(w/2)-1):
    k=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        temp.extend(ftr[i+k])
        k=k+1
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
    if len(ftr_refrmd)>50000 or i==ftr.shape[0]-m.floor(w/2)-2:
        d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
        d[-1*len(ftr_refrmd):] = ftr_refrmd
        ftr_refrmd=[]
    if i%1000==0:
        print(i)
        
for i in range(ftr.shape[0]-m.floor(w/2)-1,ftr.shape[0]):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        if j>m.floor(w/2)+(ftr.shape[0]-i)-1:
            temp.extend(np.zeros(ftr[ftr.shape[0]-1].shape))
        else:
            temp.extend(ftr[i+p+j])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
        
d.resize(d.shape[0]+len(ftr_refrmd), axis=0)   
d[-1*len(ftr_refrmd):] = ftr_refrmd
ftr_refrmd=[]

f.close()
h5f.close()


'mixed_log_30h_norm' is the normalized version of 'mixed_log_30h_nozeroinsert'. normalized by features to zero mean and unit variance.

In [6]:
file='mixed_log_30h_nozeroinsert'
file_path = os.path.normpath(os.path.join(write_path,file))
h5f = h5py.File(file_path+'.hdf5', 'r')
d=h5f[file]
tot_len=d.shape[0]
dim = d.shape[1]

lst=0
tot_num=(tot_len//10000)*10000
for i in range(0,tot_num,10000):
    mixed = h5f[file][i:i+10000]
    lst=lst+np.sum(mixed,axis=0)

mixed = h5f[file][tot_num:]
lst=lst+np.sum(mixed,axis=0)
tot_mean=lst/tot_len
lst=0
h5f = h5py.File(file_path+'.hdf5','r')
for i in range(0,tot_num,10000):
    mixed = h5f[file][i:i+10000]
    lst=lst+np.sum((mixed - tot_mean)**2,axis=0)
del mixed  

mixed = h5f[file][tot_num:]
lst=lst+np.sum((mixed - tot_mean)**2,axis=0)
lst=lst/tot_len
tot_std=np.sqrt(lst)
h5f.close()
normalized_file = 'mixed_log_30h_norm'
normalized_file_path =  os.path.normpath(os.path.join(write_path,normalized_file))
f = h5py.File(normalized_file_path+'.hdf5', 'w')
dd = f.create_dataset(normalized_file, (0,dim),maxshape=(None,dim), dtype='f', chunks=True)
h5f = h5py.File(file_path+'.hdf5','r')
for i in range(0,tot_num,10000):
    mixed = h5f[file][i:i+10000]
    scaled=(mixed-tot_mean)/tot_std
    dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
    dd[-1*scaled.shape[0]:] = scaled
del mixed    
mixed = h5f[file][tot_num:]
scaled=(mixed-tot_mean)/tot_std
dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
dd[-1*scaled.shape[0]:] = scaled
np.savetxt(os.path.normpath(os.path.join(write_path,'mean_mixed_log.txt')),tot_mean)
np.savetxt(os.path.normpath(os.path.join(write_path,'std_mixed_log.txt')),tot_std)

f.close()
h5f.close()


'ftr_refrmd_10h_norm' is the normalized(by features) of 'ftr_refrmd_10h'.

normalized to zero mean and unit variance.

In [ ]:
import h5py
import numpy as np
refrmd_file='ftr_refrmd_10h'
refrmd_file_path = os.path.normpath(os.path.join(write_path,refrmd_file))
h5f = h5py.File(refrmd_file_path+'.hdf5', 'r')
d=h5f[refrmd_file]
tot_len=d.shape[0]
dim = d.shape[1]
lst=0

for i in range(0,tot_num,10000):
    print(i)
    mixed = h5f[refrmd_file][i:i+10000]
    lst=lst+np.sum(mixed,axis=0)
tot_num=(tot_len//10000)*10000
mixed = h5f[refrmd_file][tot_num:]
lst=lst+np.sum(mixed,axis=0)
print(mixed.shape)
tot_mean=lst/tot_len
lst=0
h5f = h5py.File(refrmd_file_path+'.hdf5','r')
for i in range(0,tot_num,10000):
    mixed = h5f[refrmd_file][i:i+10000]
    lst=lst+np.sum((mixed - tot_mean)**2,axis=0)
del mixed  
mixed = h5f[refrmd_file][tot_num:]
lst=lst+np.sum((mixed - tot_mean)**2,axis=0)
lst=lst/tot_len
tot_std=np.sqrt(lst)
h5f.close()
normalized_file = 'ftr_refrmd_10h_norm'
normalized_file_path =  os.path.normpath(os.path.join(write_path,normalized_file))
f = h5py.File(normalized_file_path+'.hdf5', 'w')
dd = f.create_dataset(normalized_file, (0,dim),maxshape=(None,dim), dtype='f', chunks=True)
h5f = h5py.File(refrmd_file_path+'.hdf5','r')
for i in range(0,tot_num,10000):
    print(i)
    mixed = h5f[refrmd_file][i:i+10000]
    scaled=(mixed-tot_mean)/tot_std
    dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
    dd[-1*scaled.shape[0]:] = scaled
del mixed    
mixed = h5f[refrmd_file][tot_num:]
print(mixed.shape)
scaled=(mixed-tot_mean)/tot_std
dd.resize(dd.shape[0]+scaled.shape[0], axis=0)   
dd[-1*scaled.shape[0]:] = scaled

np.savetxt(os.path.normpath(os.path.join(write_path,'mean.txt')),tot_mean)
np.savetxt(os.path.normpath(os.path.join(write_path,'std.txt')),tot_std)

data is converted from h5py to tfrecords format.

In [7]:
import numpy as np
import tensorflow as tf
import os
__author__ = "Sangwoong Yoon"

def np_to_tfrecords(X, Y, file_path_prefix, verbose=True):
    """
    Converts a Numpy array (or two Numpy arrays) into a tfrecord file.
    For supervised learning, feed training inputs to X and training labels to Y.
    For unsupervised learning, only feed training inputs to X, and feed None to Y.
    The length of the first dimensions of X and Y should be the number of samples.
    
    Parameters
    ----------
    X : numpy.ndarray of rank 2
        Numpy array for training inputs. Its dtype should be float32, float64, or int64.
        If X has a higher rank, it should be rshape before fed to this function.
    Y : numpy.ndarray of rank 2 or None
        Numpy array for training labels. Its dtype should be float32, float64, or int64.
        None if there is no label array.
    file_path_prefix : str
        The path and name of the resulting tfrecord file to be generated, without '.tfrecords'
    verbose : bool
        If true, progress is reported.
    
    Raises
    ------
    ValueError
        If input type is not float (64 or 32) or int.
    
    """
    def _dtype_feature(ndarray):
        """match appropriate tf.train.Feature class with dtype of ndarray. """
        assert isinstance(ndarray, np.ndarray)
        dtype_ = ndarray.dtype
        if dtype_ == np.float64 or dtype_ == np.float32:
            return lambda array: tf.train.Feature(float_list=tf.train.FloatList(value=array))
        elif dtype_ == np.int64:
            return lambda array: tf.train.Feature(int64_list=tf.train.Int64List(value=array))
        else:  
            raise ValueError("The input should be numpy ndarray. \
                               Instaed got {}".format(ndarray.dtype))
            
    assert isinstance(X, np.ndarray)
    assert len(X.shape) == 2  # If X has a higher rank, 
                               # it should be rshape before fed to this function.
    assert isinstance(Y, np.ndarray) or Y is None
    
    # load appropriate tf.train.Feature class depending on dtype
    dtype_feature_x = _dtype_feature(X)
    if Y is not None:
        assert X.shape[0] == Y.shape[0]
        assert len(Y.shape) == 2
        dtype_feature_y = _dtype_feature(Y)            
    
    # Generate tfrecord writer
    result_tf_file = file_path_prefix + '.tfrecords'
    writer = tf.python_io.TFRecordWriter(result_tf_file)
    if verbose:
        print("Serializing {:d} examples into {}".format(X.shape[0], result_tf_file))
        
    # iterate over each sample,
    # and serialize it as ProtoBuf.
    for idx in range(X.shape[0]):
        x = X[idx]
        if Y is not None:
            y = Y[idx]
        
        d_feature = {}
        d_feature['X'] = dtype_feature_x(x)
        if Y is not None:
            d_feature['Y'] = dtype_feature_y(y)
            
        features = tf.train.Features(feature=d_feature)
        example = tf.train.Example(features=features)
        serialized = example.SerializeToString()
        writer.write(serialized)
    
    if verbose:
        print("Writing {} done!".format(result_tf_file))

        
#################################    
##      Test and Use Cases     ##
#################################
import h5py
load_size = 100000
k = 0
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/30hdata'
input_name = 'mixed_log_30h_norm'
input_path = os.path.normpath(os.path.join(write_path,input_name))
target_name = 'single_dataset_log_30h'
target_path = os.path.normpath(os.path.join(write_path,target_name))
parent = 'tfrecord_files'
tf_folder = 'mixed_30h_norm'
if not os.path.exists(os.path.join(Data_path,parent,tf_folder)):
    os.makedirs(os.path.join(Data_path,parent,tf_folder))
h5f1 = h5py.File(input_path+'.hdf5','r')
hh = h5py.File(target_path+'.hdf5', 'r')
d=hh[target_name]
data_len = d.shape[0]
index = np.arange(0, data_len, load_size)
for I in index:
    if data_len-I<load_size:
        xx = h5f1[input_name][I:]
        yy = hh[target_name][I:]
    else:
        xx = h5f1[input_name][I:I+load_size]
        yy = hh[target_name][I:I+load_size]
    np_to_tfrecords(xx, yy, os.path.normpath(os.path.join(Data_path,parent,tf_folder,'filenum'+str(k))), verbose=True)
    k = k+1
# 1-2. Check if the data is stored correctly
# open the saved file and check the first entries
# for serialized_example in tf.python_io.tf_record_iterator('test.tfrecords'):
#     example = tf.train.Example()
#     example.ParseFromString(serialized_example)
#     x_1 = np.array(example.features.feature['X'].float_list.value)
#     y_1 = np.array(example.features.feature['Y'].float_list.value)
#     break

# np_to_tfrecords(X, Y, file_path_prefix, verbose=True)

Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum0.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum0.tfrecords done!
Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum1.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum1.tfrecords done!
Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum2.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum2.tfrecords done!
Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_nor

Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum28.tfrecords done!
Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum29.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum29.tfrecords done!
Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum30.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum30.tfrecords done!
Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum31.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum31.tfrecor

Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum57.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum57.tfrecords done!
Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum58.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum58.tfrecords done!
Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum59.tfrecords
Writing D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_30h_norm\filenum59.tfrecords done!
Serializing 100000 examples into D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\tfrecord_files\mixed_3

In [1]:
import os
import h5py
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/30hdata'
input_name = 'mixed_log_30h_norm'
input_path = os.path.normpath(os.path.join(write_path,input_name))
target_name = 'single_dataset_log_30h'
target_path = os.path.normpath(os.path.join(write_path,target_name))
h5f1 = h5py.File(input_path+'.hdf5','r')
hh = h5py.File(target_path+'.hdf5', 'r')
d=hh[target_name]
data_len = d.shape[0]
x = h5f1[input_name]
print(x.shape)
print(d.shape)
h5f1.close()
hh.close()

(8033557, 257)
(8033557, 257)


In [1]:
import os
import h5py
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/10hdata'
input_name = 'mixed_log_10h_norm'
input_path = os.path.normpath(os.path.join(write_path,input_name))
target_name = 'single_dataset_log_10h'
target_path = os.path.normpath(os.path.join(write_path,target_name))
h5f1 = h5py.File(input_path+'.hdf5','r')
hh = h5py.File(target_path+'.hdf5', 'r')
d = hh[target_name][0:500000]
x = h5f1[input_name][0:500000]
small_path = os.path.normpath(os.path.join(write_path,'mixed_log_norm_small.h5'))
hf = h5py.File(small_path, 'w')
hf.create_dataset('mixed_log_norm_small', data=d)
hf.close()
small_path = os.path.normpath(os.path.join(write_path,'single_dataset_log_small.h5'))
hf = h5py.File(small_path, 'w')
hf.create_dataset('single_dataset_log_small', data=x)
hf.close()

## code archive

In [ ]:
mixed_logname='mixed_log_10h'
mixed_phasename='mixed_phase_10h'
mixed_log_path = os.path.normpath(os.path.join(write_path,mixed_logname))
clean_phase_path = os.path.normpath(os.path.join(write_path,mixed_phasename))
h5f = h5py.File(mixed_log_path+'.hdf5','r')
ftr = h5f[mixed_logname][0:]

ftr_refrmd=[]
ftr_refrmd_norm=[]
print('0')
for i in range(m.floor(w/2)):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        temp.extend(ftr[i+p+j])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)

print('1')
refrmd_file='ftr_refrmd_10h_norm'
refrmd_file_path = os.path.normpath(os.path.join(write_path,refrmd_file))
f = h5py.File(refrmd_file_path+'.hdf5', 'w')
d = f.create_dataset(refrmd_file, (0,257*w),maxshape=(None,257*w), dtype='f', chunks=True)
print('2')
for i in range(m.floor(w/2),ftr.shape[0]-m.floor(w/2)-1):
    k=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        temp.extend(ftr[i+k])
        k=k+1
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
    if len(ftr_refrmd)>50000 or i==ftr.shape[0]-m.floor(w/2)-2:
        print(len(ftr_refrmd))
#         for ftr_num in range(len(ftr_refrmd)):
        ftr_refrmd_norm = preprocessing.scale(ftr_refrmd, axis=0, with_mean=True, with_std=True, copy=True)
#             print(ftr_refrmd_norm.shape)
        d.resize(d.shape[0]+len(ftr_refrmd_norm), axis=0)   
        d[-1*len(ftr_refrmd_norm):] = ftr_refrmd_norm
        print(len(ftr_refrmd_norm))
        ftr_refrmd_norm=[]
        ftr_refrmd=[]
    if i%10000==0:
        print(i)
for i in range(ftr.shape[0]-m.floor(w/2)-1,ftr.shape[0]):
    p=-m.floor(w/2)
    temp = []
    for j in range(w):
        if np.all(ftr[i]==0):
            break
        if j>m.floor(w/2)+(ftr.shape[0]-i)-1:
            temp.extend(np.zeros(ftr[ftr.shape[0]-1].shape))
        else:
            temp.extend(ftr[i+p+j])
    if np.any(temp!=[]):
        ftr_refrmd.append(temp)
# for ftr_num in range(len(ftr_refrmd)):
ftr_refrmd_norm = preprocessing.scale(ftr_refrmd, axis=0, with_mean=True, with_std=True, copy=True)
d.resize(d.shape[0]+len(ftr_refrmd_norm), axis=0)   
d[-1*len(ftr_refrmd_norm):] = ftr_refrmd_norm
ftr_refrmd=[]
ftr_refrmd_norm=[]
f.close()
h5f.close()
##############################################
##############################################